<a href="https://colab.research.google.com/github/haifeng-jin/keras-benchmarking/blob/main/batch_norm_op_torch_before.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

See details in [this pull request](https://github.com/keras-team/keras/pull/18793).

This notebook runs on the base commit of the PR.

Runtime: V100

In [1]:
# %%script false --no-raise-error

!apt install -qq python3-venv
!pip install -q namex
!git clone --quiet https://github.com/haifeng-jin/keras.git
!cd keras && git checkout 66bb075f765e0c7fe80bf7af6d3cbb9148824675
!python keras/pip_build.py --install > /dev/null

The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3-venv python3.10-venv
0 upgraded, 4 newly installed, 0 to remove and 9 not upgraded.
Need to get 2,474 kB of archives.
After this operation, 2,890 kB of additional disk space will be used.
Selecting previously unselected package python3-pip-whl.
(Reading database ... 120880 files and directories currently installed.)
Preparing to unpack .../python3-pip-whl_22.0.2+dfsg-1ubuntu0.4_all.deb ...
Unpacking python3-pip-whl (22.0.2+dfsg-1ubuntu0.4) ...
Selecting previously unselected package python3-setuptools-whl.
Preparing to unpack .../python3-setuptools-whl_59.6.0-1.2ubuntu0.22.04.1_all.deb ...
Unpacking python3-setuptools-whl (59.6.0-1.2ubuntu0.22.04.1) ...
Selecting previously unselected package python3.10-venv.
Preparing to unpack .../python3.10-venv_3.10.12-1~22.04.2_amd64.deb ...
Unpa

In [2]:
import os

os.environ["KERAS_BACKEND"] = "torch"

In [4]:
import keras
import numpy as np
import torch
import time

torch.cuda.empty_cache()
keras.config.set_image_data_format("channels_first")
batch_size = 64


class BenchmarkMetricsCallback(keras.callbacks.Callback):
    def __init__(self, start_batch=1, stop_batch=None):
        self.start_batch = start_batch
        self.stop_batch = stop_batch

        self.state = {}

    def on_train_batch_begin(self, batch, logs=None):
        if batch == self.start_batch:
            self.state["benchmark_begin"] = time.time()

    def on_train_batch_end(self, batch, logs=None):
        if batch == self.stop_batch:
            self.state["benchmark_end"] = time.time()
            throughput = (self.stop_batch - self.start_batch + 1) / (
                self.state["benchmark_end"] - self.state["benchmark_begin"]
            )
            self.state["throughput"] = throughput

    def on_predict_batch_begin(self, batch, logs=None):
        if batch == self.start_batch:
            self.state["benchmark_begin"] = time.time()

    def on_predict_batch_end(self, batch, logs=None):
        if batch == self.stop_batch:
            self.state["benchmark_end"] = time.time()
            throughput = (self.stop_batch - self.start_batch + 1) / (
                self.state["benchmark_end"] - self.state["benchmark_begin"]
            )
            self.state["throughput"] = throughput


# Prepare data
steps = 101


class MyDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.images = np.random.randn(3, 224, 224)
        self.labels = np.random.randint(
            0,
            1000,
            size=tuple(),
        )

    def __len__(self):
        return steps * batch_size

    def __getitem__(self, idx):
        return self.images, self.labels


dataset = torch.utils.data.DataLoader(
    MyDataset(), batch_size=batch_size, shuffle=True
)

# Prepare the model
model = keras.applications.resnet50.ResNet50()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer="adam",
)
torch.cuda.empty_cache()

callback = BenchmarkMetricsCallback(stop_batch=100)
model.fit(dataset, epochs=1, callbacks=[callback])
print(f"training: {1000.0 / callback.state['throughput']:.0f} ms/step")

callback = BenchmarkMetricsCallback(stop_batch=100)
model.predict(dataset, callbacks=[callback])
print(f"inferencing: {1000.0 / callback.state['throughput']:.0f} ms/step")

print(f"Memory usages: {torch.cuda.memory_allocated() / 1024 / 1024} MB")

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
101/101 ━━━━━━━━━━━━━━━━━━━━ 49s 397ms/step - loss: 0.5964
training: 397 ms/step
101/101 ━━━━━━━━━━━━━━━━━━━━ 14s 142ms/step
inferencing: 141 ms/step
Memory usages: 414.5419921875 MB
